## 1.a) Extraction

In [ ]:
#Importing libraries

import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 
import requests

print("Libraries imported successfully......")

The link to all these APIs is: https://www.game-change.co.uk/2023/02/10/a-complete-guide-to-the-fantasy-premier-league-fpl-api/

Descriptions of each dataframe are in the notes.

### i. Player data Current Season


In [ ]:
### Function to return player url 
def format_player_url(player_id):
    # Define the base URL without curly brackets
    base_url = 'https://fantasy.premierleague.com/api/element-summary/{}/'
    
    # Replace the placeholder '{}' with the actual player_id
    formatted_url = base_url.format(player_id)
    
    return formatted_url

# Example usage:
player_id = 308  # Replace this with the actual player ID
formatted_url = format_player_url(player_id)
print(formatted_url)


In [ ]:
#This cell imports individual player information
r = requests.get(format_player_url(308))
player_json = r.json()


player_fixtures_df = pd.DataFrame(player_json['fixtures'])
player_history_df = pd.DataFrame(player_json['history'])
player_history_past_df = pd.DataFrame(player_json['history_past'])


In [ ]:
### This cell imports data from specified gameweek of the current season as long as the gameweek has ended or is live.
gameweek_url = 'https://fantasy.premierleague.com/api/event/2/live/'
r = requests.get(gameweek_url)
gameweek_json = r.json()

gameweek_df = pd.DataFrame(gameweek_json['elements'])


### ii. Team data

In [ ]:
# Imported full data from last seasons

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()


#Convert to df

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

### iii. Full Gameweek data 

In [ ]:
#This cell imports Full gameweek information.
fixture_url = 'https://fantasy.premierleague.com/api/fixtures/?event=2'
r = requests.get(fixture_url)
fixture_json = r.json()


fixture_df = pd.DataFrame(fixture_json)



## 1.b) Tranformation

### i. Add fixture data to the teams data

#### i.ii) Expand Team data

In [ ]:
def append_event_id(teams_df):
    # Create an empty DataFrame to store the expanded data
    expanded_df = pd.DataFrame()

    # Iterate through events from 1 to 38
    for event_id in range(1, 39):
        # Create a temporary DataFrame for the current event
        temp_df = teams_df.copy()
        
        # Set the 'event_id' column to the current event ID
        temp_df['event_id'] = event_id
        
        # Append the temporary DataFrame to the expanded DataFrame
        expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)
    
    return expanded_df

# Call the function to create an expanded DataFrame with 'event_id'
expanded_teams_df = append_event_id(teams_df)


In [36]:
import requests
import pandas as pd

def fetch_fixture_data(event_id):
    # Define the base URL with the event_id
    base_url = f'https://fantasy.premierleague.com/api/fixtures/?event={event_id}'
    
    # Replace the placeholder '{}' with the actual event_id
    formatted_url = base_url.format(event_id)
    
    # Send an HTTP GET request to the API endpoint
    response = requests.get(formatted_url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response to get fixture data
        fixture_json = response.json()
        
        # Create a DataFrame from the fixture data
        fixture_data = pd.DataFrame(fixture_json)
        
        return fixture_data
    else:
        print(f"Request failed for event {event_id} with status code {response.status_code}")
        return None  # Return None in case of a failed request


In [38]:

def append_event_id(fixture_df):
    # Create an empty DataFrame to store the expanded fixture data
    expanded_fixture_df = pd.DataFrame()

    # Iterate through events from 1 to 38
    for event_id in range(1, 39):
        # Get fixture data for the current event
        fixture_data = fetch_fixture_data(event_id)
        
        # Create a temporary DataFrame for the fixture data
        temp_df = fixture_data.copy()
        
        # Set the 'event_id' column to the current event ID
        temp_df['event_id'] = event_id
        
        # Append the temporary DataFrame to the expanded fixture DataFrame
        expanded_fixture_df = pd.concat([expanded_fixture_df, temp_df], ignore_index=True)
    
    return expanded_fixture_df

# Call the function to create an expanded fixture DataFrame with 'event_id'
expanded_fixture_df = append_event_id(fixture_df)



In [39]:
expanded_fixture_df

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,event_id
0,2367538,1,True,True,1,2023-08-11T19:00:00Z,90,False,True,13,3,6,0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,93321,1
1,2367540,1,True,True,2,2023-08-12T12:00:00Z,90,False,True,16,1,1,2,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,93322,1
2,2367539,1,True,True,3,2023-08-12T14:00:00Z,90,False,True,19,1,3,1,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93323,1
3,2367541,1,True,True,4,2023-08-12T14:00:00Z,90,False,True,12,1,5,4,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,3,93324,1
4,2367542,1,True,True,5,2023-08-12T14:00:00Z,90,False,True,10,1,9,0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,93325,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2367913,38,False,False,376,2024-05-19T15:00:00Z,0,False,False,2,None,8,None,[],3,2,93696,38
375,2367914,38,False,False,377,2024-05-19T15:00:00Z,0,False,False,20,None,11,None,[],2,4,93697,38
376,2367915,38,False,False,378,2024-05-19T15:00:00Z,0,False,False,10,None,12,None,[],2,2,93698,38
377,2367916,38,False,False,379,2024-05-19T15:00:00Z,0,False,False,19,None,13,None,[],2,5,93699,38


#### i.i) Get fixture id

In [ ]:
teams_df = teams_df.rename(columns={'name' : 'team_name', 'id':'teams_id', 'short_name' : 'team_short_name'})

In [ ]:
fixture_df = fixture_df.rename(columns={'id':'fixture_id'})

In [ ]:
def get_fixture_id(team_id):
    for idx, row in fixture_df.iterrows():
        if team_id == row['team_a'] or team_id == row['team_h']:
            return row['fixture_id']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture IDs for all teams in teams_df
teams_df['fixture_id'] = teams_df['teams_id'].apply(get_fixture_id)


In [ ]:
teams_df

#### i.ii) Get fixture difficulty

In [ ]:
def get_fixture_difficulty(team_id, fixture_id):
    for idx, row in fixture_df.iterrows():
        if fixture_id == row['fixture_id']:
            if team_id == row['team_a']:
                return row['team_a_difficulty']

            elif team_id == row['team_h']:
                return row['team_h_difficulty']
    return None  # Return None if no matching fixture is found

# Assuming you want to calculate and assign fixture difficulties for all teams in teams_df
teams_df['fixture_difficulty'] = teams_df.apply(lambda row: get_fixture_difficulty(row['teams_id'], row['fixture_id']), axis=1)


## 1.b) Loading


In [ ]:
player_history_df